In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import os
from tqdm.notebook import tqdm

In [2]:
from azure_asr import recognize_from_microphone, transcribe_audio_to_text
import pymodel

# IMPORTANT:

# change default language in azure_asr.py

In [5]:
model_name = 'google/flan-t5-small-finetuned-asr_correction-hindi'

_model, _tokenizer = pymodel.load_model(model_name)

In [6]:
def correct_asr(text, model=_model, tokenizer=_tokenizer):
    corrected = pymodel.correct(text, model, tokenizer)
    return corrected

In [8]:
import re

def remove_characters(line):
    return re.sub('[.,]', '', line)

def find_diff_words(str1, str2):
    str1 =remove_characters(str1)
    str2 =remove_characters(str2)

    words1 = str1.split()
    words2 = str2.split()
    diff_words = []
    for i in range(min((len(words1), len(words2)))):
        if words1[i].lower() != words2[i].lower():
            # print(words1[i], words2[i])
            diffs = [i, words1[i], words2[i]]
            diff_words.append(diffs)
    return diff_words

In [10]:
files = os.listdir('../data/audios/hindi/test')
files

['hin_audio_6347.wav',
 'hin_audio_6355.wav',
 'hin_audio_6396.wav',
 'hin_audio_6408.wav',
 'hin_audio_6438.wav',
 'hin_audio_6479.wav',
 'hin_audio_6486.wav',
 'hin_audio_6493.wav',
 'train']

In [15]:
df = []
for file in tqdm(files):
    if file.endswith('.wav'):
        audio_file_path = '../data/audios/hindi/test/' + file
        if os.path.exists(audio_file_path):
            print('converting')
            try:
                transcription = transcribe_audio_to_text(audio_file_path)
                corrected_transcription = correct_asr(transcription)
            except:
                print('error')

            df.append([audio_file_path, transcription, corrected_transcription])

  0%|          | 0/9 [00:00<?, ?it/s]

hello
converting


C:\Users\Hanish\AppData\Roaming\Python\Python311\site-packages\transformers\generation\utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


hello
converting
hello
converting
hello
converting
hello
converting
hello
converting
hello
converting
hello
converting


In [16]:
import pandas as pd

df = pd.DataFrame(df, columns=['file', 'azure_transcription', 'corrected_transcription'])

df.head()

,file,azure_transcription,corrected_transcription
0,../data/audios/hindi/test/hin_audio_6347.wav,Speech Recognition canceled: CancellationReaso...,Speech Recognition canceled: CancellationReaso...
1,../data/audios/hindi/test/hin_audio_6355.wav,मुझमें वो काम करने की ऊर्जा नहीं है जो मैं पहल...,
2,../data/audios/hindi/test/hin_audio_6396.wav,टेनिस खेलने के बाद मुझे अपनी गर्दन के पास एक त...,
3,../data/audios/hindi/test/hin_audio_6408.wav,"मैं कई हफ्तों से सर्दी से परेशान हूँ, लेकिन फि...",
4,../data/audios/hindi/test/hin_audio_6438.wav,"जब मैं जिम में था, तब से मेरे पेक की मांसपेशिय...",


In [19]:
recording = pd.read_csv('../data/audios/hindi/audio_file_info.csv', index_col='file_name')
# recording.index = recording['file']
recording.head()

,og_transcription
file_name,
hin_audio_0.wav,जब भी मैं उसे याद करता हूं तो मैं उदास हो जाता...
hin_audio_1.wav,जब मैं भारी चीजें उठाता हूं तो मुझे ऐसा लगता ह...
hin_audio_2.wav,जब मैं अपना हाथ हिलाता हूं तो बहुत अधिक दर्द ह...
hin_audio_3.wav,मेरे बेटे के होंठ में छेद हो गया था और वह सूज ...
hin_audio_4.wav,मेरी पीठ के निचले हिस्से की मांसपेशियाँ दर्द क...


In [21]:
correct = []
for file in df['file']:
    file = file.split('/')[-1] 

    if file in recording.index.to_list():
        correct.append(recording.loc[file]['og_transcription'])
    else:
        correct.append('NO LABEL EXISTS')

df['og_transcript'] = correct


In [22]:
differnces = []
for lin1, lin2 in zip(df['corrected_transcription'], df['og_transcript']):
    differnces.append(find_diff_words(lin1, lin2))

In [23]:
df['differences'] = differnces

In [25]:
df

,file,azure_transcription,corrected_transcription,og_transcript,differences
0,../data/audios/hindi/test/hin_audio_6347.wav,Speech Recognition canceled: CancellationReaso...,Speech Recognition canceled: CancellationReaso...,जब मैं इसे छूता हूं तो मेरे कान में दर्द होता है।,"[[0, Speech, जब], [1, Recognition, मैं], [2, c..."
1,../data/audios/hindi/test/hin_audio_6355.wav,मुझमें वो काम करने की ऊर्जा नहीं है जो मैं पहल...,,मुझमें वह काम करने की ऊर्जा नहीं है जो मैं पहल...,[]
2,../data/audios/hindi/test/hin_audio_6396.wav,टेनिस खेलने के बाद मुझे अपनी गर्दन के पास एक त...,,टेनिस खेलने के बाद मुझे अपनी गर्दन के पास एक त...,[]
3,../data/audios/hindi/test/hin_audio_6408.wav,"मैं कई हफ्तों से सर्दी से परेशान हूँ, लेकिन फि...",,मैं कई हफ्तों से सर्दी से परेशान हूं लेकिन फिर...,[]
4,../data/audios/hindi/test/hin_audio_6438.wav,"जब मैं जिम में था, तब से मेरे पेक की मांसपेशिय...",,जब मैं जिम में था तब से मेरे पेट की मांसपेशियो...,[]
5,../data/audios/hindi/test/hin_audio_6479.wav,जब मैं काफी दूर तक दौड़ा तो मुझे लगा कि मेरा द...,,जब मैं काफी दूर तक दौड़ा तो मुझे लगा कि मेरा द...,[]
6,../data/audios/hindi/test/hin_audio_6486.wav,मैंने अधिक स्पष्ट देखने के लिए अपना चश्मा लगान...,,मैंने अधिक स्पष्ट देखने के लिए अपना चश्मा लगान...,[]
7,../data/audios/hindi/test/hin_audio_6493.wav,मैं कल एक भारी बैग ले गया था और आज जब मैं उठा ...,,मैं कल एक भारी बैग ले गया था और आज जब मैं उठा ...,[]


In [26]:
df.to_csv('./tests/tested_hindi.csv', index=False)